In [1]:
import os
%pwd

'd:\\Projects\\Cancer-classfiication-using-MLflow-and-DVC\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Projects\\Cancer-classfiication-using-MLflow-and-DVC'

In [3]:
from dataclasses import dataclass
from pathlib import Path

# Creating Data Ingestion Entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.ChestCancerClassfication.constants import *
from src.ChestCancerClassfication.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        # storing the values from YAML into a  variable
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import zipfile
import gdown
from src.ChestCancerClassfication.utils import logger
from src.ChestCancerClassfication.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self) -> str:
        """ Fetch data from the URL."""

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path = str
        Extracts the zip file into the data directory
        Fanction returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-03-14 21:08:01,520: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-14 21:08:01,522: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-14 21:08:01,522: INFO: common: created directory at: artifacts]
[2024-03-14 21:08:01,522: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-14 21:08:01,522: INFO: 2261647235: Downloading data from https://drive.google.com/file/d/1sjozQle9ohHLr1bgJ_MpnM97mDt4BzFO/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1sjozQle9ohHLr1bgJ_MpnM97mDt4BzFO
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1sjozQle9ohHLr1bgJ_MpnM97mDt4BzFO&confirm=t&uuid=160cd97c-b494-44f3-a5c2-dd7d51f02e04
To: d:\Projects\Cancer-classfiication-using-MLflow-and-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:03<00:00, 34.0MB/s] 

[2024-03-14 21:08:05,940: INFO: 2261647235: Downloaded data from https://drive.google.com/file/d/1sjozQle9ohHLr1bgJ_MpnM97mDt4BzFO/view?usp=sharing into file artifacts/data_ingestion/data.zip]
